In [8]:
from web3 import Web3
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import time

COLLECTION_NAME = "sss"
COLLECTION_FOLDER = "SSS"

ENDPOINT = ""
w3 = Web3(Web3.HTTPProvider(ENDPOINT))
WEI_TO_GWEI_CONSTANT = 1e9 # to GWEI
START_BLOCK = 13276866
END_BLOCK = 13276877
EDGE_BLOCKS = 100
MAX_BLOCK_PULL = 1024

first_block = START_BLOCK - EDGE_BLOCKS
last_block = END_BLOCK + EDGE_BLOCKS


blocks_left = last_block - first_block + 1
blocks_to_grab = min(MAX_BLOCK_PULL, blocks_left)
block_list = []
print("Total Blocks: ", blocks_left)

while blocks_left > 0:
    blocks = w3.eth.fee_history(blocks_to_grab, last_block, [10, 90]) # args: block_count, newest_block,
    last_block = blocks["oldestBlock"]
    
    blocks_to_grab = min(MAX_BLOCK_PULL, last_block - first_block + 1)
    blocks_left -= len(blocks["baseFeePerGas"][:-1]) 
    
    new_start_of_list = blocks["baseFeePerGas"][:-1]
    new_start_of_list.extend(block_list[1:])
    block_list = new_start_of_list

print("Total Length of List: ", len(block_list))
print("first base fee: ", block_list[0] / WEI_TO_GWEI_CONSTANT) 
print("last base fee: ", block_list[-1] / WEI_TO_GWEI_CONSTANT) 

# get the timestamp for every block
block_numbers = list(range(first_block, END_BLOCK + EDGE_BLOCKS + 1)) 
time_list = []
for i in range(0, len(block_list)):
    time_list.append(int(w3.eth.getBlock(block_numbers[i]).timestamp)) 
    block_list[i] = block_list[i] / WEI_TO_GWEI_CONSTANT
    

data = pd.DataFrame()
data["block_numbers"] = block_numbers
data["timestamps"] = time_list
data["timestamps"]=(pd.to_datetime(data["timestamps"], unit="s")) 
data["base_fee"] =  block_list

data.to_csv("../../{}/data/gas_data.csv".format(COLLECTION_NAME))

Total Blocks:  212
Total Length of List:  212
first base fee:  89.024973653
last base fee:  106.795220785
